# Cellpose segmentation

Notebook showing how to segment cells using Cellpose.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from skimage.io import imsave
import tensorflow as tf
import cellpose
from cellpose import models
from cellpose import utils

sys.path.append('../libraries')
import input_functions as inp_f

In [2]:
info_file_path = r'Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\info_B02_128_188.txt'

In [3]:
#Add stuff to do GPU segmentation

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
#Is cellpose using the GPU, reports True or False
cellpose.core.use_gpu(gpu_number=0, use_torch=True)

True

In [4]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:36:24_Pacific_Standard_Time_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0
Tue Sep 19 12:50:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 529.08       Driver Version: 529.08       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P620        WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P0    N/A /  N/A |     40MiB /  4096MiB |      0%      Default |
|                    

In [7]:
# load cellpose model - set gpu to False if your machine does not support CUDA based GPU segmentation
model = models.Cellpose(gpu=True, model_type='cyto')

In [8]:
# read the file
info_file = open(info_file_path, 'r')
info_lines = info_file.readlines()
info_file.close()

# read info about the data frame
exp_dir,df_name = inp_f.read_df_info(info_lines)

# get info about the channels
channel_list = inp_f.read_channels(info_lines)

In [9]:
# read the movie to segment

file_name = [x['file_name'] for x in channel_list if x['tracking']][0]
c = [x['channel_in_file'] for x in channel_list if x['tracking']][0]
im_path = os.path.join(exp_dir,'data',file_name)
im_path

im = inp_f.open_movie(im_path,c)

In [10]:
# check how many timepoints are there in the file
frames_num = im.shape[0]

print(f'Total frame number: {frames_num}')

Total frame number: 61


In [11]:
# loop for segmentation - This is the default, and assumes you can load the entire image into memory at once

frames_num = 64
for i in range(0,frames_num): # it's a small example - just 4 first frames
    
    # get an image
    im_frame = im[i,:,:]

    # segment the right plane
    labels, _, _, _ = model.eval(im_frame, diameter=30, channels=[0,0])

    # save segmentation
    save_dir = os.path.join(exp_dir,'segmentation')
    save_file = file_name[:-4]+f'_{str(i).zfill(3)}_label.png'
    save_path = os.path.join(save_dir,save_file)
    imsave(save_path,labels.astype('uint16')) 

RuntimeError: CUDA out of memory. Tried to allocate 1.35 GiB (GPU 0; 4.00 GiB total capacity; 2.08 GiB already allocated; 0 bytes free; 2.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [11]:
import torch
import numpy as np

def split_into_patches(image, patch_size):
    patches = []
    for i in range(0, image.shape[0], patch_size):
        for j in range(0, image.shape[1], patch_size):
            patch = image[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
    return patches

def stitch_patches(patches, original_shape, patch_size):
    image = np.empty(original_shape)
    patch_idx = 0
    for i in range(0, original_shape[0], patch_size):
        for j in range(0, original_shape[1], patch_size):
            image[i:i+patch_size, j:j+patch_size] = patches[patch_idx]
            patch_idx += 1
    return image

frames_num = 61
patch_size = 7000  # or whatever size fits in your memory - measured in pixels
max_label = 0  # To store the maximum label ID to maintain count across patches
for i in range(0,frames_num): 
    
    im_frame = im[i,:,:]
    patches = split_into_patches(im_frame, patch_size)
    segmented_patches = []

    for patch in patches:
        labels, _, _, _ = model.eval(patch, diameter=30, channels=[0,0])
        labels[labels > 0] += max_label  # Add max_label only to the identified nuclei
        segmented_patches.append(labels)
        max_label = np.max(labels)  # Update max_label for the next patch

    segmented_image = stitch_patches(segmented_patches, im_frame.shape, patch_size)

    save_dir = os.path.join(exp_dir,'segmentation')
    save_file = file_name[:-4]+f'_{str(i).zfill(3)}_label.png'
    save_path = os.path.join(save_dir,save_file)
    imsave(save_path,segmented_image.astype('uint16')) 

    torch.cuda.empty_cache()
    max_label = 0


C:\Users\gases\.conda\envs\livecell_GPU_segment\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\segmentation\WellB02_ChannelCFP_Seq0000 - Stitched_128_188_000_label.png is a low contrast image
C:\Users\gases\.conda\envs\livecell_GPU_segment\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\segmentation\WellB02_ChannelCFP_Seq0000 - Stitched_128_188_001_label.png is a low contrast image
C:\Users\gases\.conda\envs\livecell_GPU_segment\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\segmentation\WellB02_ChannelCFP_Seq0000 - Stitched_128_188_002_label.png is a low contrast image
C:\Users\gases\.conda\envs\livecell_GPU_segment\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\segmentation\WellB02_ChannelCFP_Seq0000 - Stitched_128_188_003_label.png is a low contrast image
C:\Users\gases\.conda\en

C:\Users\gases\.conda\envs\livecell_GPU_segment\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\segmentation\WellB02_ChannelCFP_Seq0000 - Stitched_128_188_034_label.png is a low contrast image
C:\Users\gases\.conda\envs\livecell_GPU_segment\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\segmentation\WellB02_ChannelCFP_Seq0000 - Stitched_128_188_035_label.png is a low contrast image
C:\Users\gases\.conda\envs\livecell_GPU_segment\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\segmentation\WellB02_ChannelCFP_Seq0000 - Stitched_128_188_036_label.png is a low contrast image
C:\Users\gases\.conda\envs\livecell_GPU_segment\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\segmentation\WellB02_ChannelCFP_Seq0000 - Stitched_128_188_037_label.png is a low contrast image
C:\Users\gases\.conda\en